In [1]:
from predict import predict_labels
from wettbewerb import load_references, save_predictions
import time
from score import score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pywt
from sklearn import tree
from sklearn import preprocessing
from IPython.display import display
from ecgdetectors import Detectors
import tensorflow as tf
from tensorflow import keras
import filter_ecgToTest
from filter_ecgToTest import filter
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

In [90]:
ecg_leads, ecg_labels, fs, ecg_names = load_references('training')


6000	 Dateien wurden geladen.


In [104]:
#preprocess the data and rename the labels into integers
for idx, ecg_lead in enumerate(ecg_leads):
    ecg_leads[idx]=filter(ecg_leads[idx])
    ecg_leads[idx]=ecg_lead/np.amax(ecg_leads[idx])
    
for idx, label in enumerate(ecg_labels):
    if label=="N":
        ecg_labels[idx]=0
    if label=="A":
        ecg_labels[idx]=1
    if label=="O":
        ecg_labels[idx]=2
    if label=="~":
        ecg_labels[idx]=3
        

In [105]:
#split into training and validation datasets
ecg_train, ecg_test, labels_train, labels_test,names_train,names_test = train_test_split(ecg_leads, 
                                                                  ecg_labels,ecg_names,
                                                                  test_size=0.2, shuffle =True,stratify=ecg_labels)

In [201]:
#finds the optimal value for the signal segmentation length from the traing dataset
import statistics
def find_optimal_length(hb):
    opt_len=list()
    for idx, ecg in enumerate(hb):
        opt_len.append(len(ecg))
    opt_len=statistics.median(opt_len)
    #opt_len=mode(opt_len)
    return opt_len

In [48]:
import neurokit2 as nk
import pandas as pd
from ecgdetectors import Detectors
def split_HB(num_HB, ecg, labels,names,fs):
    
    #this split the data in segments of 3 heart beats
    hb=list()
    hb_labels=list()
    #this saves which split signal is part from which actual whole signal
    decoder=list()
    #start detectors as alternative
    detectors=Detectors(fs)
    for idx, ecg_lead in enumerate(ecg):
         #converts to a series
        ecg_lead=pd.Series(ecg_lead)
        #find the R peaks
        try:
            _, rpeaks = nk.ecg_peaks(ecg_lead,sampling_rate=fs)
            rpeaks=rpeaks['ECG_R_Peaks']
            prev_slice=rpeaks[0]

        #if we cant find the r peaks because sadge, then use other detector
        except:
            rpeaks = detectors.pan_tompkins_detector(ecg_lead)
            prev_slice=rpeaks[0]
            print("┬┬﹏┬┬")
            #this is the splitting of the signal
        for count, posR in enumerate(rpeaks):
            #only extract from the 3d peak
            if (count%num_HB)==0 and len(ecg_lead[prev_slice:posR])>0:
                hb.append(ecg_lead[prev_slice:posR])
                hb_labels.append(labels[idx])
                decoder.append(names[idx])
                prev_slice=posR
    return hb, hb_labels,decoder
    

In [282]:
from imblearn.over_sampling import SMOTE 
from sklearn.decomposition import PCA
#numHB is an integer that gives the number of beats each signals will have from the original signal
def generate_data_set(numHB,ecg,labels,names,freq,smote:bool=False,minL:int=0,pca_prev=None,pca:bool=False):
    #data will be returen as a dictionary with the desired variables 
    data={}
    #first get the ecg split into hear beats
    hb,hb_labels,decoder=split_HB(numHB,ecg,labels,names,freq)
    #find the optimal length if minL was not given
    if minL<=0:
        minL=int(find_optimal_length(hb))
        
     #transforms it into a list of np array
    temp=list()
    for idx,ecg_lead in enumerate(hb): 
        temp.append(ecg_lead.to_numpy())
    
    #make all heart beat signals the same length, for larger signal cut them, for shorter signals just add 0s until it matches the desired length
  
    #new approach to delete the other signals that are too small
    new_ecg=list()
    new_labels=list()
    #instead of removing them, we want to do something different, add them in another list
    for idx, ecg_lead in enumerate(temp):
        if len(ecg_lead)>=minL:
            new_ecg.append(ecg_lead[:minL])
            new_labels.append(hb_labels[idx])
            #new_ecg_names.append(ecg_names[idx])
        else:
            toAdd=minL-len(ecg_lead)
            toAdd=np.zeros(toAdd)
            #merges the signal that was too short with signal full of 0 to achive the desired lenght
            new_ecg.append(np.concatenate((ecg_lead, toAdd)))
            new_labels.append(hb_labels[idx])
            #new_ecg_names.append(ecg_names[idx])
    if smote:
        sm = SMOTE()
        new_ecg, new_labels = sm.fit_resample(new_ecg, new_labels)

    
    #only use pca if we say so
    if pca:
        #if we dont send any older pca, then create its own
        if pca_prev is None:
            pca_prev=PCA(.99)
            pca_prev.fit(new_ecg)
        #use the pca to reduce the dimension of the data
        new_ecg=pca_prev.transform(new_ecg)
        data['pca']=pca_prev

 #now convert the data into working dataset for the cnn model
    x=new_ecg
    x=np.array(x)
    x=x.reshape((x.shape[0],x.shape[1],1))
    
    y=np.array(new_labels)
    
    #return the data in a dictionary way, to simplily later use of the desired variables
   
    data['input']=x
    data['labels']=y
    data['decoder']=decoder
    data['optimal_length']=minL
    
    return data

In [18]:
model_dir="cnn_1D"
model_dir=os.path.join(os.getcwd(),model_dir)
pred_dir=model_dir

### Generate Data

In [294]:
train=generate_data_set(3,ecg_train,labels_train,names_train,fs,
                        smote=True,
                        pca=False
                       )
#train is a dictionary, get the actual variables
x_train=train['input']
y_train=train['labels']
opt_length=train['optimal_length']
try:
    pca_train=train['pca']
except:
    pca_train=None
print("----------")
test=generate_data_set(3,ecg_test,labels_test,names_test,fs,minL=opt_length
                      ,pca_prev=pca_train
                       ,pca=False
                      )
x_test=test['input']
y_test=test['labels']
decoder_test=test['decoder']


┬┬﹏┬┬
┬┬﹏┬┬
┬┬﹏┬┬
----------


In [295]:

#CNN
from tensorflow.keras.layers import Flatten, Dense, Conv1D, MaxPool1D, Dropout
# Create sequential model 
cnn_model = tf.keras.models.Sequential()
#First CNN layer  with 32 filters, conv window 3, relu activation and same padding
cnn_model.add(Conv1D(filters=32, kernel_size=(3,), strides=1, padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001), input_shape = (x_train.shape[1:])))
#Second CNN layer  with 64 filters, conv window 3, relu activation and same padding
cnn_model.add(Conv1D(filters=64, kernel_size=(3,), strides=1, padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
#Third CNN layer with 128 filters, conv window 3, relu activation and same padding
cnn_model.add(Conv1D(filters=128, kernel_size=(3,), strides=1,padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
#Fourth CNN layer with Max pooling
cnn_model.add(MaxPool1D(pool_size=(3,), strides=2, padding='same'))
cnn_model.add(Dropout(0.5))
#Flatten the output
cnn_model.add(Flatten())
#Add a dense layer with 256 neurons
cnn_model.add(Dense(units = 64, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
#Add a dense layer with 512 neurons
cnn_model.add(Dense(units = 128, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
#Add a dense layer with 512 neurons
cnn_model.add(Dense(units = 256, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
#Softmax as last layer with five outputs
cnn_model.add(Dense(units = 4, activation='softmax'))


In [296]:
opt=tf.keras.optimizers.RMSprop(learning_rate=0.0001)
cnn_model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
cnn_model.summary()

Model: "sequential_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_99 (Conv1D)           (None, 715, 32)           128       
_________________________________________________________________
conv1d_100 (Conv1D)          (None, 715, 64)           6208      
_________________________________________________________________
conv1d_101 (Conv1D)          (None, 715, 128)          24704     
_________________________________________________________________
max_pooling1d_33 (MaxPooling (None, 358, 128)          0         
_________________________________________________________________
dropout_33 (Dropout)         (None, 358, 128)          0         
_________________________________________________________________
flatten_33 (Flatten)         (None, 45824)             0         
_________________________________________________________________
dense_132 (Dense)            (None, 64)              

In [297]:
epochs = 3
batch_size = 32
weights = {0: 1.,1: 10.,2: 1.,3: 1}
from keras import callbacks
earlystopping = callbacks.EarlyStopping(monitor ="val_loss", mode ="min", patience = 5, restore_best_weights = True)
history = cnn_model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data = (x_test, y_test),
class_weight=weights,
#callbacks =[earlystopping]
)

Epoch 1/3
4299/4299 [==============================] - 24s 5ms/step - loss: 1.4150 - accuracy: 0.5744 - val_loss: 1.3992 - val_accuracy: 0.4761
Epoch 2/3
4299/4299 [==============================] - 24s 5ms/step - loss: 0.8511 - accuracy: 0.7305 - val_loss: 1.2191 - val_accuracy: 0.5578
Epoch 3/3
4299/4299 [==============================] - 24s 6ms/step - loss: 0.6755 - accuracy: 0.7762 - val_loss: 1.1657 - val_accuracy: 0.5901


In [298]:
import statistics
from statistics import mode
#get the predictions
classes = cnn_model.predict(x_test)
predictions=list()
for idx, ecg_lead in enumerate (classes):
    predictions.append(np.argmax(ecg_lead))

#this dictionary will save all the predictions in groups
label_decoder={}
for idx, label in enumerate(predictions):
    if decoder_test[idx] in label_decoder:
        label_decoder[decoder_test[idx]].append(label)
    else:
        label_decoder[decoder_test[idx]]=list()
        label_decoder[decoder_test[idx]].append(label)
#to get the most frequent prediction for each signal 
labels_toTest=list()
labels_toSave=list()
for idx,keys in enumerate(label_decoder):
    #get the most frequent prediction
    frequent_label=mode(label_decoder[keys])
    #convert it into a letter
    labels2give="N"
    if int(frequent_label)==0:
        labels2give="N"
    elif int(frequent_label)==1:
        labels2give="A"
    elif int(frequent_label)==2:
        labels2give="O"
    else:
        labels2give="~"
    #save the final predictred labels
    labels_toTest.append((keys,labels2give))

for i,ecg in enumerate(labels_test):
    ecg_class=int(ecg)
    label2give="N"
    if int(ecg_class)==0:
        labels2give="N"
    elif int(ecg_class)==1:
        labels2give="A"
    elif int(ecg_class)==2:
        labels2give="O"
    else:
        labels2give="~"
    #esto es para el name
    labels_toSave.append((names_test[i],labels2give))

save_predictions(labels_toTest,model_dir,"PREDICTIONS")
save_predictions(labels_toSave,model_dir,"REFERENCE")
score(model_dir)

1200	 Labels wurden geschrieben.
1200	 Labels wurden geschrieben.


(0.8495575221238938,
 0.5014962786669622,
 {'N': {'n': 625, 'a': 26, 'o': 51, 'p': 14},
  'A': {'n': 6, 'a': 96, 'o': 1, 'p': 1},
  'O': {'n': 154, 'a': 109, 'o': 70, 'p': 10},
  'P': {'n': 10, 'a': 10, 'o': 5, 'p': 12}})